In [ ]:
cap = cv2.VideoCapture(0)#capture video, 0 means built-in webcam
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False, #静态追踪，低于0.5置信度会再一次跟踪
                      max_num_hands=2, # 最多有2只手
                      min_detection_confidence=0.5, # 最小检测置信度
                      min_tracking_confidence=0.5)  # 最小跟踪置信度
mpDraw = mp.solutions.drawing_utils
pTime = 0 #处理一张图像前的时间
cTime = 0 #一张图处理完的时间
# 存放坐标信息
lmList = []
captured_coordinates = []

#使用数据库保存得到的坐标
conn = sqlite3.connect('hand_coordinates.db')
# 创建一个表
db = conn.cursor()
# 创建用于保存归一化坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS normalized_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x REAL,
    y REAL
);
''')

# 创建用于保存像素坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS pixel_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x INTEGER,
    y INTEGER
);
''')
conn.commit()
# 初始化
cap = cv2.VideoCapture(0)


normalized_coords = []
pixel_coords = []
lmList = []  
pTime = 0

# 创建保存数据的目录
output_dir = "output_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

while True:
    # 视频读取和预处理
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    print(results.multi_hand_landmarks)

    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            for index, lm in enumerate(handlms.landmark):
                if index == 8:  # 如果你只关心索引为8的关键点
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    print(index, cx, cy)
                    lmList.append([index, cx, cy])
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

                    # 插入数据到数据库
                    db.execute("INSERT INTO normalized_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, lm.x, lm.y))

                    # 将像素坐标保存到数据库
                    db.execute("INSERT INTO pixel_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, cx, cy))

                    conn.commit()

                    # 在图像上画圆
                    cv2.circle(img, (cx, cy), 12, (0, 220, 23), cv2.FILLED)

            # 其他代码（绘制关键点等）

    # FPS显示和其他后处理
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

    cv2.imshow('Image', img)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# 关闭数据库连接
conn.close()
# 释放摄像头资源
cap.release()
# 关闭所有 OpenCV 窗口
cv2.destroyAllWindows()
#娴熟数据
conn = sqlite3.connect('hand_coordinates.db')
db = conn.cursor()
# 查询数据库中的数据
db.execute("SELECT * FROM normalized_coordinates")

# 获取查询结果
rows = db.fetchall()

# 打印查询结果
for row in rows:
    print(row)
